In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from textblob import TextBlob
import re

In [ ]:
with open('applai_dataa.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['applai']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [ ]:
training_sentences

['hello',
 'hi',
 'hey',
 'greetings',
 'what`s up?',
 'good day',
 'bye',
 'bye bye',
 'good bye',
 'I`m leaving',
 'I will go now',
 'what is your name?',
 'how should I call you ?',
 'what`s your name?',
 'who are you?',
 'Can you tell me your name?',
 'when is applai`s training held?',
 'when is the training held?',
 'how many trainings are held?',
 'Does applai have any trainings?',
 'where is applai`s training held?',
 'where could I find applai`s training ?',
 'at what place is the training held?',
 'how many departments are there in applai ?',
 'what are applai`s departments ?',
 'where is applai`s training held?',
 'where could I find applai`s training ?',
 'at what place is the training held?',
 'Does applai hold any campaigns ?',
 'what other activities does applai offer?',
 'When was applai club established?',
 'For how many years has applai been working?',
 'When was applai created at Ain Shams University?',
 'Does applai have any other branches?',
 'How many branches does

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
____________________________________________________

In [ ]:

epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 16ms/step - loss: 2.1972 - accuracy: 0.1026
Epoch 2/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1959 - accuracy: 0.1538
Epoch 3/500
2/2 [==============================] - 0s 10ms/step - loss: 2.1948 - accuracy: 0.1538
Epoch 4/500
2/2 [==============================] - 0s 9ms/step - loss: 2.1940 - accuracy: 0.0769
Epoch 5/500
2/2 [==============================] - 0s 10ms/step - loss: 2.1931 - accuracy: 0.0769
Epoch 6/500
2/2 [==============================] - 0s 10ms/step - loss: 2.1923 - accuracy: 0.0256
Epoch 7/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1915 - accuracy: 0.0769
Epoch 8/500
2/2 [==============================] - 0s 9ms/step - loss: 2.1906 - accuracy: 0.1538
Epoch 9/500
2/2 [==============================] - 0s 16ms/step - loss: 2.1899 - accuracy: 0.1795
Epoch 10/500
2/2 [==============================] - 0s 14ms/step - loss: 2.1890 - accuracy: 0.1795
Epoch 11/500
2/2 [=====

In [ ]:
model.save('applai_chatbot_model.h5',history)

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)